# 🧪 Swin Transformer 모델 단위 테스트

이 노트북은 Swin Transformer 모델의 동작을 테스트합니다:
- 모델 빌드 및 초기화 확인
- 추천 모델 선택 기능 테스트
- 샘플 데이터 추론 및 출력 검증
- GPU/CPU 모드 자동 설정 확인

**테스트 항목:**
- 설정 파일 로드 및 모델 파라미터 확인
- Swin 모델 생성 및 가중치 로딩
- 더미 입력을 통한 추론 테스트
- 출력 형태 및 값 범위 검증
- 모델 파라미터 수 계산

In [1]:
# [1] 프로젝트 루트 디렉토리로 이동
import os                                                   # OS 모듈 임포트
import sys                                                  # 시스템 모듈 임포트
os.chdir("../../../")                                       # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())                      # 현재 디렉토리 출력

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [ ]:
# [2] 폰트 설정 및 경고 억제
# 경고 억제 설정
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 적용 및 시각화 환경 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔고딕 폰트 경로 및 설정
font_path = './font/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 폰트 등록 및 설정 (한글 텍스트 표시를 위함)
fe = fm.FontEntry(fname=font_path, name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams['font.family'] = 'NanumGothic'      # 기본 폰트를 나눔고딕으로 설정
plt.rcParams['font.size'] = 10                   # 기본 글자 크기 설정
plt.rcParams['axes.unicode_minus'] = False       # 마이너스 기호 깨짐 방지

# 글자 겹침 방지를 위한 레이아웃 설정
plt.rcParams['figure.autolayout'] = True         # 자동 레이아웃 조정
plt.rcParams['axes.titlepad'] = 20               # 제목과 축 사이 여백

# 폰트 로드 확인
try:
    test_font = fm.FontProperties(fname=font_path)
    print("✅ 나눔고딕 폰트 로드 성공")
except Exception as e:
    print(f"❌ 폰트 로드 실패: {e}")

In [3]:
# [3] 라이브러리 및 로거 설정
# 필요한 라이브러리 및 유틸리티 모듈 임포트
import torch                                                # PyTorch 딥러닝 프레임워크
import numpy as np                                          # 수치 계산 라이브러리
from src.models.build import build_model, get_recommended_model  # 모델 빌드 및 추천 함수
from src.utils.common import load_yaml                      # 설정 로드 함수
from src.logging.notebook_logger import create_notebook_logger   # 노트북 로거 생성 함수 임포트

# 단위 테스트 로거 초기화
logger = create_notebook_logger(
    base_log_dir="modular",
    folder_name="unit_tests",
    file_name="03_swin_model_test"
)

print('✅ 라이브러리 및 로거 설정 완료')

📝 노트북 작업 시작: 03_swin_model_test
📝 로그 디렉토리: notebooks/modular/unit_tests/03_swin_model_test/20250907_080613
✅ 라이브러리 및 로거 설정 완료


In [4]:
# [4] GPU 자동 체크
# GPU 사용 가능 여부 확인
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

✅ GPU 사용 가능: NVIDIA GeForce RTX 4090


In [5]:
# [5] 설정 파일 로드 및 모델 이름 확인
# 설정 파일에서 모델 정보 불러오기
try:
    cfg = load_yaml("configs/train_highperf.yaml")  # 설정 파일 로드
    
    # 추천 모델 이름 가져오기
    model_name = get_recommended_model(cfg['model']['name'])
    print(f'모델 이름: {cfg["model"]["name"]} -> {model_name}')
    print(f'클래스 수: {cfg["data"]["num_classes"]}')
    print(f'이미지 크기: {cfg["train"]["img_size"]}')
    print('✅ 설정 파일 로드 완료')
except Exception as e:
    print(f'⚠️ 설정 파일 로드 실패: {e}')

모델 이름: swin_base_384 -> swin_base_patch4_window12_384_in22k
클래스 수: 17
이미지 크기: 384
✅ 설정 파일 로드 완료


In [6]:
# [6] Swin 모델 생성
# 설정에 따른 Swin 모델 빌드
try:
    model = build_model(
        name=model_name,                        # 추천 모델 이름 사용
        num_classes=cfg['data']['num_classes'], # 클래스 수
        pretrained=cfg['model']['pretrained'],  # pretrained 가중치 사용
        drop_rate=cfg['model']['drop_rate'],    # dropout 비율
        drop_path_rate=cfg['model']['drop_path_rate']  # stochastic depth 비율
    )
    print(f'✅ 모델 생성 완료: {model_name}')
    print(f'모델 총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}')
    
    # GPU로 모델 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print('✅ 모델을 GPU로 이동 완료')
    else:
        print('ℹ️ CPU에서 모델 실행')
        
except Exception as e:
    print(f'⚠️ 모델 생성 실패: {e}')

✅ 모델 생성 완료: swin_base_patch4_window12_384_in22k
모델 총 파라미터 수: 86,896,009
✅ 모델을 GPU로 이동 완료


In [7]:
# [7] 샘플 추론 테스트
# 더미 입력으로 모델 추론 테스트
try:
    # 더미 입력 생성
    dummy_input = torch.randn(1, 3, cfg['train']['img_size'], cfg['train']['img_size'])
    print(f'더미 입력 shape: {dummy_input.shape}')
    
    # GPU 사용 시 입력도 GPU로 이동
    if torch.cuda.is_available():
        dummy_input = dummy_input.cuda()
        print('✅ 입력을 GPU로 이동 완료')
    
    # 샘플 추론 실행
    with torch.no_grad():  # 그래디언트 계산 비활성화
        output = model(dummy_input)
    
    print(f'✅ 샘플 추론 결과 shape: {output.shape}')
    print(f'출력 값 범위: [{output.min().item():.4f}, {output.max().item():.4f}]')
    
except Exception as e:
    print(f'⚠️ 샘플 추론 실패: {e}')

더미 입력 shape: torch.Size([1, 3, 384, 384])
✅ 입력을 GPU로 이동 완료
✅ 샘플 추론 결과 shape: torch.Size([1, 17])
출력 값 범위: [-0.5979, 0.3639]
✅ 샘플 추론 결과 shape: torch.Size([1, 17])
출력 값 범위: [-0.5979, 0.3639]


In [8]:
# [8] 테스트 결과 저장 및 완료
# 모든 테스트 결과를 정리하여 저장
try:
    # 테스트 결과 딕셔너리 생성
    result = {
        'model_name': cfg['model']['name'],
        'actual_model_name': model_name,
        'num_classes': cfg['data']['num_classes'],
        'input_shape': list(dummy_input.shape),
        'output_shape': list(output.shape),
        'total_params': sum(p.numel() for p in model.parameters()),
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'output_min': float(output.min().item()),
        'output_max': float(output.max().item())
    }
    
    # 테스트 결과 저장 (notebook_logger API 사용)
    logger.save_results('swin_model_test_result', result)
    logger.finalize()
    
    print('✅ 모든 테스트 완료!')
    print(f'📁 결과 저장 위치: {logger.base_dir}')
    print('✅ Swin 모델 테스트 성공적으로 완료')
    
except Exception as e:
    print(f'⚠️ 결과 저장 실패: {e}')

📝 결과 저장: swin_model_test
✅ 작업 완료! 총 소요 시간: 1.61초
✅ 결과 요약: notebooks/modular/unit_tests/03_swin_model_test/20250907_080613/summary.json

🏁 노트북 작업 완료: 03_swin_model_test
📁 결과 디렉토리: notebooks/modular/unit_tests/03_swin_model_test/20250907_080613
⏱️ 소요 시간: 1.61초
📊 섹션 수: 1
✅ 모든 테스트 완료!
📁 결과 저장 위치: notebooks/modular/unit_tests/03_swin_model_test/20250907_080613
✅ Swin 모델 테스트 성공적으로 완료


In [9]:
# [9] 저장된 테스트 결과 확인
# summary.json 파일 내용 출력 및 디렉토리 구조 확인
try:
    from pathlib import Path
    import json
    
    # summary.json 파일 경로 (notebook_logger는 summary.json 사용)
    summary_path = Path(logger.base_dir) / 'summary.json'
    
    print("📁 결과 저장 디렉토리:")
    print(f"   {logger.base_dir}")
    print()
    
    # 저장된 파일들 확인
    if Path(logger.base_dir).exists():
        print("📄 저장된 파일 목록:")
        for item in Path(logger.base_dir).rglob("*"):
            if item.is_file():
                relative_path = item.relative_to(logger.base_dir)
                file_size = item.stat().st_size
                print(f"   📄 {relative_path} ({file_size:,} bytes)")
        print()
    
    # summary.json 파일 내용 출력
    if summary_path.exists():
        with open(summary_path, 'r', encoding='utf-8') as f:
            summary_data = json.load(f)
        
        print("📄 테스트 요약 내용:")
        print(json.dumps(summary_data, indent=2, ensure_ascii=False))
    else:
        print(f"⚠️ 요약 파일을 찾을 수 없습니다: {summary_path}")
        
except Exception as e:
    print(f'⚠️ 결과 확인 실패: {e}')

📁 결과 저장 디렉토리:
   notebooks/modular/unit_tests/03_swin_model_test/20250907_080613

📄 저장된 파일 목록:
   📄 summary.json (835 bytes)
   📄 results/swin_model_test_result.json (410 bytes)
   📄 logs/03_swin_model_test_20250907_080613.log (627 bytes)

📄 테스트 요약 내용:
{
  "file_name": "03_swin_model_test",
  "folder_name": "unit_tests",
  "start_time": "2025-09-07T08:06:13.088090",
  "end_time": "2025-09-07T08:06:14.697638",
  "duration_seconds": 1.609548,
  "total_sections": 1,
  "results": {
    "swin_model_test": {
      "model_name": "swin_base_384",
      "actual_model_name": "swin_base_patch4_window12_384_in22k",
      "num_classes": 17,
      "input_shape": [
        1,
        3,
        384,
        384
      ],
      "output_shape": [
        1,
        17
      ],
      "total_params": 86896009,
      "device": "cuda",
      "output_min": -0.597861647605896,
      "output_max": 0.36391207575798035,
      "timestamp": "2025-09-07T08:06:14.697138",
      "section": "swin_model_test"
    }
  }

---

## 📋 **테스트 결과 요약**

### ✅ **검증된 기능**
- **모델 빌드**: Swin Transformer 모델 생성 및 초기화
- **설정 로드**: YAML 설정 파일에서 모델 파라미터 로딩
- **추천 모델**: 설정에 따른 최적 모델 변형 자동 선택
- **GPU 지원**: CUDA 사용 가능 시 GPU로 모델 자동 이동
- **추론 테스트**: 더미 입력을 통한 순전파 검증

### 📊 **주요 결과**
- **모델 이름**: 설정된 모델과 실제 사용된 모델명
- **파라미터 수**: 총 학습 가능한 파라미터 개수
- **입출력 형태**: 입력 텐서와 출력 텐서의 shape
- **출력 범위**: 모델 출력값의 최솟값과 최댓값
- **디바이스**: GPU 또는 CPU 사용 여부

### 🔍 **검증 방법**
- **기능 테스트**: 각 단계별 정상 동작 여부 확인
- **형태 검증**: 입출력 텐서 차원 및 크기 점검
- **수치 안정성**: 출력값이 적절한 범위 내에 있는지 확인
- **로그 분석**: summary.json에서 모델 정보 및 성능 지표 검토

### 💡 **문제 해결 가이드**
- **모델 로드 실패**: pretrained 가중치 다운로드 상태 확인
- **CUDA 메모리 부족**: 배치 크기 축소 또는 모델 크기 조정
- **설정 파일 오류**: YAML 파일 문법 및 경로 확인
- **추론 실패**: 입력 데이터 형태 및 디바이스 일치 여부 점검
- **성능 저하**: 모델 파라미터 및 하이퍼파라미터 조정

### 📈 **성능 최적화 팁**
- **모델 경량화**: 작은 변형 모델(Swin-T) 사용으로 속도 향상
- **Mixed Precision**: AMP 사용으로 메모리 절약 및 가속화
- **배치 최적화**: GPU 메모리에 맞는 최적 배치 크기 설정
- **컴파일 최적화**: torch.compile() 사용으로 추론 속도 향상